In [281]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium 

In [2]:
file=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

## Creating a DataFrame from the Wikipedia data

In [3]:
neighbourhood=pd.DataFrame(file[0])
neighbourhood.columns=['PostCode','Borough','Neighbourhood']
neighbourhood.drop(0,inplace=True)
neighbourhood.head()

,PostCode,Borough,Neighbourhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights


In [4]:

# Removing the rows where values are not assigned
neighbourhood.drop(neighbourhood[neighbourhood['Neighbourhood']=='Not assigned'].index,inplace=True)
neighbourhood.drop(neighbourhood[neighbourhood['Borough']=='Not assigned'].index,inplace=True)

In [5]:
geolocator=Nominatim(user_agent='foursquare_agent')

In [6]:
lat=[]
long=[]
index=[]
for i,val in neighbourhood.iterrows():
    location=str(val['Neighbourhood'])+', Toronto'
    
    try:
        address=geolocator.geocode(location)
        lat.append(address.latitude)
        long.append(address.longitude)
    except:
        try:
            address=geolocator.geocode(location)
            lat.append(address.latitude)
            long.append(address.longitude)
        except:
            index.append(i)

In [7]:
neighbourhood.drop(index,inplace=True)

In [8]:
neighbourhood['Latitude']=lat
neighbourhood['Longitude']=long

In [320]:
neighbourhood.shape

(200, 5)

## Visualizing the neighborhoods on Map


In [294]:
latT=43.6532
lonT=-79.382
map=folium.Map(location=[latT,lonT],zoom_start=12)

folium.CircleMarker(
                        [latT,lonT],
                        label=lab,
                        radius=5,
                        color='blue',fill=True,fillcolor='blue',fillopacity=0.3).add_to(map)
for lat,long,lab in zip(neighbourhood['Latitude'],neighbourhood['Longitude'],neighbourhood['Neighbourhood']):
    folium.CircleMarker(
                        [lat,long],
                        label=lab,
                        radius=5,
                        color='red',fill=True,fillcolor='red',fillopacity=0.3).add_to(map)
map

## Exploring places near each neighbourhood by using Foursquare API

In [190]:
#CLIENT_ID = 'JFGH3PREVZK0JAEXJ4WANVT0C1XNAHHUC4NHYTBBA2OR2P0W' # your Foursquare ID
#CLIENT_SECRET = 'L1JMFW35UOM0TZBH3BGXQOT1DGTZ5OL5EIGQPJ11BTAMVHUY' # your Foursquare Secret
#CLIENT_SECRET = 'L1JMFW35UOM0TZBH3BGXQOT1DGTZ5OL5EIGQPJ11BTAMVHUY' # your Foursquare Secret
CLIENT_ID = 'L114DB3JQMLRT50I0KUF3CAL2FC4TGJJDLHKK2LAVHG0WNME' # your Foursquare ID
CLIENT_SECRET = 'LEA2NYG2145BM2SQAQJGWX43CELE2QG1UUE2VFNMTXG0ZBPI' # your Foursquare Secret
version = '20180604'
limit = 50
query='Indian'
radius=10000


In [252]:
num=0
common_venue=pd.DataFrame(columns=['Location','1st Common Venue','2nd Common Venue','3rd Common Venue','4th Common Venue','5th Common Venue','6th Common Venue','7th Common Venue','8th Common Venue','9th Common Venue','10th Common Venue'] )
cnames=common_venue.columns

for enum in neighbourhood.iterrows():
    if num<2:
        latitude=enum[1]['Latitude']
        longitude=enum[1]['Longitude']
        location=enum[1]['Neighbourhood']
    # Creating Foursquare search for all the neighbourhoods
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        version, 
        str(latitude), 
        str(longitude), 
        radius, 
        limit)
    

        response=requests.get(url).json()
        length=len(response['response']['groups'][0]['items'])
        print(length,' popular destinations near ', location)

        venue=[]
        category=[]
        for i in range(0,length):
            vname=response['response']['groups'][0]['items'][i]['venue']['name']
            cat=response['response']['groups'][0]['items'][i]['venue']['categories'][0]['name']
            venue.append(vname)
            category.append(cat)
        df=pd.DataFrame()
        df['venue']=venue
        df['Category']=category

        summary=df.groupby(df['Category']).count()
        summary=pd.DataFrame(summary)
        sort=pd.DataFrame(summary.sort_values(by='venue',ascending=False))
        



        data=[str(location)]
        j=0
        for i,venue in enumerate(sort['Category'].index):
            try:
                temp_cv[cnames[i+1]]=venue
                data.append(venue)
            except:
                j=j+1
        temp_cv=pd.DataFrame([data],columns=cnames)
        common_venue=common_venue.append(temp_cv)

50  popular destinations near  Parkwoods
50  popular destinations near  Victoria Village
50  popular destinations near  Harbourfront
50  popular destinations near  Lawrence Heights
50  popular destinations near  Lawrence Manor
50  popular destinations near  Queen's Park
50  popular destinations near  Rouge
50  popular destinations near  Malvern
50  popular destinations near  Don Mills North
50  popular destinations near  Woodbine Gardens
50  popular destinations near  Parkview Hill
50  popular destinations near  Ryerson
50  popular destinations near  Garden District
50  popular destinations near  Glencairn
50  popular destinations near  Cloverdale
50  popular destinations near  Islington
50  popular destinations near  Martin Grove
50  popular destinations near  Princess Gardens
50  popular destinations near  West Deane Park
50  popular destinations near  Highland Creek
50  popular destinations near  Rouge Hill
50  popular destinations near  Port Union
50  popular destinations near  Fle

50  popular destinations near  Old Mill North
50  popular destinations near  Church and Wellesley
50  popular destinations near  Humber Bay
50  popular destinations near  King's Mill Park
50  popular destinations near  Kingsway Park South East
50  popular destinations near  Mimico NE
50  popular destinations near  Old Mill South
50  popular destinations near  The Queensway East
50  popular destinations near  Royal York South East
50  popular destinations near  Sunnylea
50  popular destinations near  Kingsway Park South West
50  popular destinations near  Mimico NW
50  popular destinations near  The Queensway West
50  popular destinations near  Royal York South West
50  popular destinations near  South of Bloor


In [253]:
common_venue.shape

(200, 11)

In [275]:
common_venue_copy=common_venue.copy()

In [276]:
cat=common_venue_copy['Location']
common_venue_copy.drop('Location',axis=1,inplace=True)

In [277]:
Encoded=pd.get_dummies(common_venue_copy)

In [278]:
Encoded.shape


(200, 526)

In [279]:
Encoded.head()

,1st Common Venue_Bakery,1st Common Venue_Beach,1st Common Venue_Bookstore,1st Common Venue_Brewery,1st Common Venue_Café,1st Common Venue_Caribbean Restaurant,1st Common Venue_Coffee Shop,1st Common Venue_Fast Food Restaurant,1st Common Venue_Golf Course,1st Common Venue_Grocery Store,1st Common Venue_Hotel,1st Common Venue_Italian Restaurant,1st Common Venue_Japanese Restaurant,1st Common Venue_Korean Restaurant,1st Common Venue_Middle Eastern Restaurant,1st Common Venue_Park,1st Common Venue_Sandwich Place,1st Common Venue_Steakhouse,1st Common Venue_Supermarket,1st Common Venue_Zoo Exhibit,2nd Common Venue_Bakery,2nd Common Venue_Bar,2nd Common Venue_Beer Bar,2nd Common Venue_Brewery,2nd Common Venue_Burger Joint,2nd Common Venue_Café,2nd Common Venue_Caribbean Restaurant,2nd Common Venue_Chinese Restaurant,2nd Common Venue_Cocktail Bar,2nd Common Venue_Coffee Shop,2nd Common Venue_Cosmetics Shop,2nd Common Venue_Farmers Market,2nd Common Venue_Golf Course,2nd Common Venue_Grocery Store,2nd Common Venue_Gym,2nd Common Venue_Hotel,2nd Common Venue_Indian Restaurant,2nd Common Venue_Italian Restaurant,2nd Common Venue_Japanese Restaurant,2nd Common Venue_Korean Restaurant,2nd Common Venue_Liquor Store,2nd Common Venue_Middle Eastern Restaurant,2nd Common Venue_Park,2nd Common Venue_Pizza Place,2nd Common Venue_Restaurant,2nd Common Venue_Sandwich Place,2nd Common Venue_Scenic Lookout,2nd Common Venue_Seafood Restaurant,2nd Common Venue_Sporting Goods Shop,2nd Common Venue_Steakhouse,2nd Common Venue_Supermarket,2nd Common Venue_Thai Restaurant,2nd Common Venue_Vegetarian / Vegan Restaurant,2nd Common Venue_Warehouse Store,2nd Common Venue_Yoga Studio,2nd Common Venue_Zoo Exhibit,3rd Common Venue_American Restaurant,3rd Common Venue_Art Gallery,3rd Common Venue_Asian Restaurant,3rd Common Venue_Bagel Shop,3rd Common Venue_Bakery,3rd Common Venue_Bar,3rd Common Venue_Beach,3rd Common Venue_Brewery,3rd Common Venue_Burger Joint,3rd Common Venue_Burrito Place,3rd Common Venue_Café,3rd Common Venue_Caribbean Restaurant,3rd Common Venue_Chinese Restaurant,3rd Common Venue_Clothing Store,3rd Common Venue_Cocktail Bar,3rd Common Venue_Coffee Shop,3rd Common Venue_Comic Shop,3rd Common Venue_Concert Hall,3rd Common Venue_Diner,3rd Common Venue_Farmers Market,3rd Common Venue_Furniture / Home Store,3rd Common Venue_Gastropub,3rd Common Venue_Greek Restaurant,3rd Common Venue_Grocery Store,3rd Common Venue_Gym,3rd Common Venue_Gym / Fitness Center,3rd Common Venue_Hotel,3rd Common Venue_Indian Restaurant,3rd Common Venue_Italian Restaurant,3rd Common Venue_Japanese Restaurant,3rd Common Venue_Liquor Store,3rd Common Venue_Middle Eastern Restaurant,3rd Common Venue_Noodle House,3rd Common Venue_Park,3rd Common Venue_Pub,3rd Common Venue_Restaurant,3rd Common Venue_Sandwich Place,3rd Common Venue_Seafood Restaurant,3rd Common Venue_Sporting Goods Shop,3rd Common Venue_Steakhouse,3rd Common Venue_Supermarket,3rd Common Venue_Thai Restaurant,3rd Common Venue_Theater,3rd Common Venue_Yoga Studio,3rd Common Venue_Zoo Exhibit,4th Common Venue_American Restaurant,4th Common Venue_Art Gallery,4th Common Venue_Asian Restaurant,4th Common Venue_BBQ Joint,4th Common Venue_Bagel Shop,4th Common Venue_Bakery,4th Common Venue_Bar,4th Common Venue_Beach,4th Common Venue_Brewery,4th Common Venue_Bubble Tea Shop,4th Common Venue_Burger Joint,4th Common Venue_Café,4th Common Venue_Caribbean Restaurant,4th Common Venue_Cocktail Bar,4th Common Venue_Coffee Shop,4th Common Venue_Comic Shop,4th Common Venue_Deli / Bodega,4th Common Venue_Diner,4th Common Venue_Eastern European Restaurant,4th Common Venue_Farmers Market,4th Common Venue_French Restaurant,4th Common Venue_Gastropub,4th Common Venue_Grocery Store,4th Common Venue_Hakka Restaurant,4th Common Venue_Historic Site,4th Common Venue_Hookah Bar,4th Common Venue_Hotel,4th Common Venue_Indian Restaurant,4th Common Venue_Japanese Restaurant,4th Common Venue_Liquor Store,4th Common Venue_Mediterranean Res

In [283]:
km = KMeans(n_clusters=6)
km.fit(Encoded)

clusters=km.labels_



## Adding the cluster to existing Dataframe of Neighbourhoods

In [323]:
common_venue['Cluster']=clusters
common_venue['Latitude']=neighbourhood['Latitude'].values
common_venue['Longitude']=neighbourhood['Longitude'].values
common_venue

,Location,Cluster,Latitude,Longitude,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue,6th Common Venue,7th Common Venue,8th Common Venue,9th Common Venue,10th Common Venue
0,Parkwoods,0,43.757846,-79.315975,Supermarket,Middle Eastern Restaurant,Italian Restaurant,Bakery,Burger Joint,Gym / Fitness Center,Grocery Store,Caribbean Restaurant,Arts & Crafts Store,Other Great Outdoors
0,Victoria Village,1,43.732658,-79.311189,Park,Café,Middle Eastern Restaurant,Bakery,Brewery,Afghan Restaurant,Italian Restaurant,Liquor Store,Movie Theater,Other Great Outdoors
0,Harbourfront,1,43.640080,-79.380150,Park,Hotel,Theater,Café,Coffee Shop,Sporting Goods Shop,Japanese Restaurant,Gym,Farmers Market,American Restaurant
0,Lawrence Heights,3,43.722778,-79.450933,Café,Park,Indian Restaurant,Bar,Grocery Store,Gastropub,Coffee Shop,Restaurant,Middle Eastern Restaurant,Juice Bar
0,Lawrence Manor,3,43.722079,-79.437507,Grocery Store,Park,Café,Indian Restaurant,BBQ Joint,Bar,Restaurant,Coffee Shop,Gastropub,Juice Bar
0,Queen's Park,3,43.659659,-79.390340,Café,Vegetarian / Vegan Restaurant,Concert Hall,Art Gallery,Restaurant,Sandwich Place,Cosmetics Shop,Thai Restaurant,Theater,Sporting Goods Shop
0,Rouge,4,43.804930,-79.165837,Park,Caribbean Restaurant,Zoo Exhibit,Mexican Restaurant,Gym,Coffee Shop,Indian Restaurant,Burger Joint,Beach,Zoo
0,Malvern,2,43.809196,-79.221701,Zoo Exhibit,Park,Caribbean Restaurant,Bakery,Coffee Shop,Zoo,Indian Restaurant,Hakka Restaurant,Clothing Store,Community Center
0,Don Mills North,0,43.775347,-79.345944,Middle Eastern Restaurant,Caribbean Restaurant,Chinese Restaurant,Steakhouse,Japanese Restaurant,Italian Restaurant,Supermarket,Shopping Plaza,Shopping Mall,Sushi Restaurant
0,Woodbine Gardens,1,43.712078,-79.302567,Park,Brewery,Beach,Café,Greek Restaurant,Middle Eastern Restaurant,Pizza Place,Turkish Restaurant,Shopping Mall,Soccer Field


In [324]:
common_venue=common_venue[['Location', 'Cluster','Latitude', 'Longitude','1st Common Venue', '2nd Common Venue', '3rd Common Venue',
       '4th Common Venue', '5th Common Venue', '6th Common Venue',
       '7th Common Venue', '8th Common Venue', '9th Common Venue',
       '10th Common Venue']]

## Exploring the clusters

In [325]:
c1=common_venue[common_venue['Cluster']==0]
c1

,Location,Cluster,Latitude,Longitude,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue,6th Common Venue,7th Common Venue,8th Common Venue,9th Common Venue,10th Common Venue
0,Parkwoods,0,43.757846,-79.315975,Supermarket,Middle Eastern Restaurant,Italian Restaurant,Bakery,Burger Joint,Gym / Fitness Center,Grocery Store,Caribbean Restaurant,Arts & Crafts Store,Other Great Outdoors
0,Don Mills North,0,43.775347,-79.345944,Middle Eastern Restaurant,Caribbean Restaurant,Chinese Restaurant,Steakhouse,Japanese Restaurant,Italian Restaurant,Supermarket,Shopping Plaza,Shopping Mall,Sushi Restaurant
0,Ryerson,0,43.658469,-79.378993,Coffee Shop,Farmers Market,Café,Hotel,Theater,Thai Restaurant,Japanese Restaurant,Gym,Restaurant,Pizza Place
0,Garden District,0,43.656500,-79.377114,Coffee Shop,Farmers Market,Café,Hotel,Park,Theater,Gym,Sandwich Place,Neighborhood,Pizza Place
0,Cloverdale,0,43.633637,-79.549745,Coffee Shop,Bakery,Burger Joint,Brewery,Restaurant,Seafood Restaurant,Sporting Goods Shop,Skating Rink,Pool Hall,Pub
0,Martin Grove,0,43.653963,-79.387207,Sandwich Place,Hotel,Theater,Café,Art Gallery,Coffee Shop,Gym,Cosmetics Shop,Pizza Place,Restaurant
0,Don Mills South,0,43.775347,-79.345944,Middle Eastern Restaurant,Caribbean Restaurant,Chinese Restaurant,Steakhouse,Japanese Restaurant,Italian Restaurant,Supermarket,Shopping Plaza,Shopping Mall,Sushi Restaurant
0,Bloordale Gardens,0,43.635317,-79.563674,Coffee Shop,Bakery,Burger Joint,Seafood Restaurant,Sporting Goods Shop,Eastern European Restaurant,Sandwich Place,Arts & Crafts Store,Pub,Restaurant
0,Eringate,0,43.662273,-79.576516,Steakhouse,Golf Course,Japanese Restaurant,Hotel,Sandwich Place,Sporting Goods Shop,Eastern European Restaurant,Coffee Shop,Café,Burger Joint
0,Markland Wood,0,43.631239,-79.585434,Coffee Shop,Sporting Goods Shop,Bakery,Grocery Store,Sandwich Place,Seafood Restaurant,Eastern European Restaurant,Japanese Restaurant,Burrito Place,Sports Bar


In [326]:
c2=common_venue[common_venue['Cluster']==1]
c2

,Location,Cluster,Latitude,Longitude,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue,6th Common Venue,7th Common Venue,8th Common Venue,9th Common Venue,10th Common Venue
0,Victoria Village,1,43.732658,-79.311189,Park,Café,Middle Eastern Restaurant,Bakery,Brewery,Afghan Restaurant,Italian Restaurant,Liquor Store,Movie Theater,Other Great Outdoors
0,Harbourfront,1,43.640080,-79.380150,Park,Hotel,Theater,Café,Coffee Shop,Sporting Goods Shop,Japanese Restaurant,Gym,Farmers Market,American Restaurant
0,Woodbine Gardens,1,43.712078,-79.302567,Park,Brewery,Beach,Café,Greek Restaurant,Middle Eastern Restaurant,Pizza Place,Turkish Restaurant,Shopping Mall,Soccer Field
0,Parkview Hill,1,43.706298,-79.321907,Park,Café,Coffee Shop,Brewery,Pizza Place,American Restaurant,Liquor Store,Hotel,Indie Movie Theater,Italian Restaurant
0,Islington,1,43.645335,-79.524816,Bakery,Café,Coffee Shop,Burger Joint,Ice Cream Shop,Brewery,Restaurant,Park,Sporting Goods Shop,Sandwich Place
0,Princess Gardens,1,43.640466,-79.391224,Park,Italian Restaurant,Hotel,Yoga Studio,Sporting Goods Shop,Sandwich Place,Scenic Lookout,Performing Arts Venue,Record Shop,Restaurant
0,Rouge Hill,1,43.780271,-79.130499,Park,Gym,Coffee Shop,Zoo,Restaurant,Mexican Restaurant,Indian Restaurant,Zoo Exhibit,Beach,Campground
0,Port Union,1,43.775504,-79.134976,Park,Gym,Coffee Shop,Zoo,Restaurant,Mexican Restaurant,Indian Restaurant,Zoo Exhibit,Beach,Campground
0,Flemingdon Park,1,43.718432,-79.333204,Park,Café,Coffee Shop,Brewery,Pizza Place,Juice Bar,Historic Site,History Museum,Hotel,Indie Movie Theater
0,Woodbine Heights,1,43.699930,-79.319132,Park,Café,Coffee Shop,Brewery,Pizza Place,American Restaurant,Mediterranean Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant


In [327]:
c3=common_venue[common_venue['Cluster']==2]
c3

,Location,Cluster,Latitude,Longitude,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue,6th Common Venue,7th Common Venue,8th Common Venue,9th Common Venue,10th Common Venue
0,Malvern,2,43.809196,-79.221701,Zoo Exhibit,Park,Caribbean Restaurant,Bakery,Coffee Shop,Zoo,Indian Restaurant,Hakka Restaurant,Clothing Store,Community Center
0,West Deane Park,2,43.663199,-79.568568,Golf Course,Burger Joint,Coffee Shop,Steakhouse,Japanese Restaurant,Bakery,Café,Sandwich Place,Eastern European Restaurant,Sporting Goods Shop
0,St. James Town,2,43.669403,-79.372704,Coffee Shop,Park,Farmers Market,Hotel,Middle Eastern Restaurant,Comic Shop,Diner,Japanese Restaurant,Pizza Place,Liquor Store
0,Old Burnhamthorpe,2,43.639373,-79.584387,Grocery Store,Sporting Goods Shop,Coffee Shop,Eastern European Restaurant,Bakery,Sandwich Place,Seafood Restaurant,Japanese Restaurant,Steakhouse,Supermarket
0,Trinity,2,43.665302,-79.395252,Café,Thai Restaurant,Coffee Shop,Comic Shop,Concert Hall,Sandwich Place,Restaurant,Art Gallery,Record Shop,Music School
0,East Birchmount Park,2,43.714167,-79.271109,Brewery,Park,Beach,Middle Eastern Restaurant,American Restaurant,Pizza Place,Indian Restaurant,Bakery,Asian Restaurant,Café
0,Bayview Village,2,43.769197,-79.376662,Korean Restaurant,Park,Japanese Restaurant,Bakery,Bubble Tea Shop,Café,Caribbean Restaurant,Middle Eastern Restaurant,Steakhouse,Restaurant
0,Riverdale,2,43.665470,-79.352594,Coffee Shop,Park,Diner,Historic Site,Bakery,Middle Eastern Restaurant,Farmers Market,Italian Restaurant,Hotel,Pizza Place
0,India Bazaar,2,43.672223,-79.323503,Coffee Shop,Park,Café,Brewery,Italian Restaurant,Farmers Market,Historic Site,Pizza Place,Bakery,Performing Arts Venue
0,Victoria Hotel,2,43.669469,-79.373268,Coffee Shop,Park,Farmers Market,Diner,Hotel,Middle Eastern Restaurant,Japanese Restaurant,Comic Shop,Steakhouse,Liquor Store


In [328]:
c4=common_venue[common_venue['Cluster']==3]
c4

,Location,Cluster,Latitude,Longitude,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue,6th Common Venue,7th Common Venue,8th Common Venue,9th Common Venue,10th Common Venue
0,Lawrence Heights,3,43.722778,-79.450933,Café,Park,Indian Restaurant,Bar,Grocery Store,Gastropub,Coffee Shop,Restaurant,Middle Eastern Restaurant,Juice Bar
0,Lawrence Manor,3,43.722079,-79.437507,Grocery Store,Park,Café,Indian Restaurant,BBQ Joint,Bar,Restaurant,Coffee Shop,Gastropub,Juice Bar
0,Queen's Park,3,43.659659,-79.390340,Café,Vegetarian / Vegan Restaurant,Concert Hall,Art Gallery,Restaurant,Sandwich Place,Cosmetics Shop,Thai Restaurant,Theater,Sporting Goods Shop
0,Glencairn,3,43.708712,-79.440685,Café,Bar,Park,Grocery Store,Indian Restaurant,Restaurant,Coffee Shop,Gastropub,Beer Bar,Seafood Restaurant
0,Christie,3,43.664111,-79.418405,Café,Cocktail Bar,Indian Restaurant,Park,Bakery,Bar,Beer Bar,Brewery,Tapas Restaurant,Restaurant
0,Wilson Heights,3,43.740519,-79.440017,Café,Bakery,Park,Coffee Shop,Italian Restaurant,Clothing Store,Tea Room,Furniture / Home Store,Bookstore,Liquor Store
0,Dovercourt Village,3,43.665307,-79.432636,Café,Beer Bar,Indian Restaurant,Cocktail Bar,Pizza Place,Bar,Brewery,Tapas Restaurant,Restaurant,Park
0,Dufferin,3,43.660202,-79.435719,Café,Cocktail Bar,Park,Restaurant,Tapas Restaurant,Beer Bar,Brewery,Coffee Shop,Indian Restaurant,Pizza Place
0,Little Portugal,3,43.647413,-79.431116,Café,Pizza Place,Park,Cocktail Bar,Tapas Restaurant,Bakery,Bar,Beer Bar,Japanese Restaurant,Korean Restaurant
0,Parkdale Village,3,43.640495,-79.436897,Café,Park,Cocktail Bar,Pizza Place,Bakery,Bar,Beer Bar,Tapas Restaurant,Ramen Restaurant,Korean Restaurant


In [329]:
c5=common_venue[common_venue['Cluster']==5]
c5

,Location,Cluster,Latitude,Longitude,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue,6th Common Venue,7th Common Venue,8th Common Venue,9th Common Venue,10th Common Venue
0,Downsview North,5,43.749299,-79.462248,Korean Restaurant,Restaurant,Thai Restaurant,Middle Eastern Restaurant,Café,Grocery Store,Sushi Restaurant,Coffee Shop,Supermarket,Park
0,Downsview East,5,43.749299,-79.462248,Korean Restaurant,Restaurant,Thai Restaurant,Middle Eastern Restaurant,Café,Grocery Store,Sushi Restaurant,Coffee Shop,Supermarket,Park
0,Downsview West,5,43.749299,-79.462248,Korean Restaurant,Restaurant,Thai Restaurant,Middle Eastern Restaurant,Café,Grocery Store,Sushi Restaurant,Coffee Shop,Supermarket,Park
0,Downsview,5,43.749299,-79.462248,Korean Restaurant,Restaurant,Thai Restaurant,Middle Eastern Restaurant,Café,Grocery Store,Sushi Restaurant,Coffee Shop,Supermarket,Park
0,Downsview Central,5,43.749299,-79.462248,Korean Restaurant,Restaurant,Thai Restaurant,Middle Eastern Restaurant,Café,Grocery Store,Sushi Restaurant,Coffee Shop,Supermarket,Park
0,Downsview Northwest,5,43.749299,-79.462248,Korean Restaurant,Restaurant,Thai Restaurant,Middle Eastern Restaurant,Café,Grocery Store,Sushi Restaurant,Coffee Shop,Supermarket,Park


### Visualizing the groups on Map

In [332]:
latT=43.6532
lonT=-79.382
map=folium.Map(location=[latT,lonT],zoom_start=10)


for lat,long,lab in zip(c1['Latitude'],c1['Longitude'],c1['Location']):
    folium.CircleMarker(
                        [lat,long],
                        label=lab,
                        radius=5,
                        color='red',fill=True,fillcolor='red',fillopacity=0.3).add_to(map)
    
for lat,long,lab in zip(c2['Latitude'],c2['Longitude'],c2['Location']):
    folium.CircleMarker(
                        [lat,long],
                        label=lab,
                        radius=5,
                        color='gold',fill=True,fillcolor='gold',fillopacity=0.3).add_to(map)
    
for lat,long,lab in zip(c3['Latitude'],c3['Longitude'],c3['Location']):
    folium.CircleMarker(
                        [lat,long],
                        label=lab,
                        radius=5,
                        color='black',fill=True,fillcolor='black',fillopacity=0.3).add_to(map)
    
for lat,long,lab in zip(c4['Latitude'],c4['Longitude'],c4['Location']):
    folium.CircleMarker(
                        [lat,long],
                        label=lab,
                        radius=5,
                        color='green',fill=True,fillcolor='green',fillopacity=0.3).add_to(map)
    
for lat,long,lab in zip(c5['Latitude'],c5['Longitude'],c5['Location']):
    folium.CircleMarker(
                        [lat,long],
                        label=lab,
                        radius=5,
                        color='blue',fill=True,fillcolor='blue',fillopacity=0.3).add_to(map)
map